In [1]:
import numpy as np
import pandas as pd
import networkx as nx


transaction_dataset_path = '../0to999999_NormalTransaction'
transaction_dataframe = pd.read_csv(transaction_dataset_path)
# block_dataframe = pd.read_csv(block_dataset_path)

In [2]:
tr_filtered = transaction_dataframe[(transaction_dataframe['blockNumber'] > 46147) & (transaction_dataframe['blockNumber'] < 46647)]
tr_filtered.to_csv(path_or_buf='0to499_NormalTransaction',index=False)

In [4]:
tr = tr_filtered['gasPrice'].max() * 0.8
tr_filtered[tr_filtered['gasPrice'] > tr]

,blockNumber,timestamp,transactionHash,from,to,creates,value,gasLimit,gasPrice,gasUsed,callingFunction,isError
5,46214,1438919394,0xe17d4d0c4596ea7d5166ad5da600a6fdc49e26e06801...,0xa1e4380a3b1f749673e270229993ee55f35663b4,0xc9d4035f4a9226d50f79b73aafb5d874a1b6537e,None,31337,21750,50000000000000,21748,0x74796d34,None
7,46219,1438919461,0xe891897177614c91284b6929dc2ada5a87705c4729bd...,0xa1e4380a3b1f749673e270229993ee55f35663b4,0x5df9b87991262f6ba471f09758cde1c0fc1de734,None,31337,21800,50000000000000,21748,0x74796d34,None
20,46253,1438920022,0xfaa38b22af568040ad5f21db3d361736e7c94ee64f74...,0x762998e1d75227fced7a70be109a4c0b4ed86414,0x9e0afd592b1e40bee26816e360f02eebf04ad74b,None,1000000000000000,21000,50000000000000,21000,0x,None


In [10]:
import copy

class Atom:

	def __init__(self, id, terms):
		self._id = id
		self._terms = terms

	def getId(self):
		return self._id

	def get_terms(self):
		return self._terms

	def is_mapped(self, atom):
		result = False
		aux_result = True and self._id == atom.getId()
		if (len(self._terms) == len(atom.get_terms())):
			for x in range(0, len(self._terms)):
				aux_result = aux_result and (self._terms[x].getValue() == atom.get_terms()[x].getValue() or (not self._terms[x].isInstanced()))
			result = aux_result

		return result

	def get_mapping(self, atom):
		result = {}
		if self.is_mapped(atom):
			for x in range(0, len(self._terms)):
				if (not self._terms[x].isInstanced()):
					result[self._terms[x].getId()] = atom.get_terms()[x]
				elif (not atom.get_terms()[x].isInstanced()):
					result[atom.get_terms()[x].getId()] = self._terms[x]
		
		return result

	def map(self, mapping):
		success = True
		for i in range(0, len(self._terms)):
			otherTerm = mapping.get(self._terms[i].getId())
			if (otherTerm != None):
				if (self._terms[i].can_be_instanced()):
					self._terms.remove(self._terms[i])
					self._terms.insert(i, otherTerm)
				elif (not self._terms[i].getValue() == otherTerm.getValue()):
					success = False
		return success

	def __str__(self):
		result = self._id
		for term in self._terms:
			result = result + '\n' +str(term)

		return result

	def __eq__(self, atom):
		result = self._id == atom.getId()
		if len(self._terms) == len(atom.get_terms()):
			for x in range(0, len(self._terms)):
				result = result and self._terms[x] == atom.get_terms()[x]
		else:
			result = False

		result = result or (self is atom)
		
		return result

	def __hash__(self):
		string = str(self._id)
		for term in self._terms:
			string = string + str(term)

		return hash(string)

	def is_equivalent(self, atom):
		result = True
		if (self._id == atom.getId()) and (len(self._terms) == len(atom.get_terms())):
			aux_terms = copy.deepcopy(atom.get_terms())
			index = 0
			for term in self._terms:
				if (not (term.getValue() == aux_terms[index].getValue())):
					result = False
					break
				index += 1					
		else:
			result = False


		return result
    

from Ontological.Term import Term

class Constant(Term):

	def __init__(self, value):
		super().__init__(value, value)


In [12]:
transaction_dataframe = transaction_dataframe.drop(columns=['timestamp', 'callingFunction'])
transaction_dataframe = transaction_dataframe[transaction_dataframe["isError"] == 'None']
transaction_dataframe = transaction_dataframe[transaction_dataframe['to'] != 'None']

KeyError: "['timestamp' 'callingFunction'] not found in axis"

In [16]:
addresses = transaction_dataframe['from'].unique()
print(addresses)
for address, tx in addresses.iterrows():
    print(Atom('Account', Constant(address)))

['0xa1e4380a3b1f749673e270229993ee55f35663b4'
 '0xbd08e0cddec097db7901ea819a3d1fd9de8951a2'
 '0x63ac545c991243fa18aec41d4f6f598e555015dc' ...
 '0x348477e8e200349374e3d07bbb1ba8a5609aeb75'
 '0x0061a8d8280432270f19a32b6b86e5ff2d029fed'
 '0xc91d33e08a4f9a28e36c2ad2997c6a2c90ef400f']


AttributeError: 'numpy.ndarray' object has no attribute 'iterrows'

In [14]:
tx_aux = transaction_dataframe[transaction_dataframe['blockNumber'] == 999999]
_orm = []
for index, tx in tx_aux.iterrows():
    _orm.append(Atom('GasPrice', [Constant(tx['blockNumber']), Constant(tx['gasPrice'])]))

In [52]:
_orm

In [20]:
# blocks(address) : Conj<blockNumber>
# inDegree(address, blockNumber) : num
# outDegree(address, blockNumber) : num
# transacciones(address) : Conj<(Transacciones, blockNumber)>
# transacciones(address, blockNumber) : Conj<Transacciones>
# gasPrice(tx, address, blockNumber) : num
# numTransacciones(address, blockNumber) : num
# balance(address, blockNumber) : num
# vecinos(address, blockNumber) : Conj<nodo>
# contratosCreados(address) : Conj<address>
# invocacionesDe(address) : Conj<tx> (data != null)
# invocacionesA(address) : Conj<tx> (data != null)
# esInvocacion(tx) : bool
# esTransferenciaDeValor(tx) : bool
# esCreacionDeContrato(tx) : bool
# value(tx) : num
# esContrato(address) : bool
# esEOA(address) : bool
# diferenciaDeTransacciones(address, blockNumber, blockNumber) : num


## Auxiliary functions

## Returns transaction sent FROM address
def out_transactions(df, address):
    assert len(address) == 42, 'address provided is incorrect'
    return df[df['from']==address]

## Returns transaction sent TO address
def in_transactions(df, address):
    assert len(address) == 42, 'address provided is incorrect'
    return df[df['to']==address]

## Returns transaction sent TO address in block number blockNumber
def in_transaction_per_block(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    df_aux = in_transactions(df,address)
    return df_aux[df_aux['blockNumber']==blockNumber]

## Returns transaction sent FROM address in block number blockNumber
def out_transaction_per_block(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    df_aux = out_transactions(df,address)
    return df_aux[df_aux['blockNumber']==blockNumber]

## Returns transaction sent FROM and TO address in block number blockNumber
def transaction_per_block(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    return df[((df['to']==address) | (df['from']==address)) & (df['blockNumber']==blockNumber)]
    

# R1 - Temporal Burst: Si la diferencia de transacciones en un nodo en los tiempos t1 y t2 es mayor que \theta, ese nodo es sospechoso 

# diferenciaDeTransacciones(A, B1, B2, DIF) & (DIF > t_tx) & (B1 < B2) --> hyp_malicioso(A)  


#Returns the number of transactions in blockFrom - number of transactions in blockTo both sent TO address
def in_transactionsDifferenceBetweenBlocks(df, address, blockFrom, blockTo):
    assert blockFrom < blockTo, 'blockFrom should be lower'
    assert len(address) == 42, 'address provided is incorrect'
    return in_degree(in_transaction_per_block(df, address, blockFrom), address, blockFrom) - in_degree(in_transaction_per_block(df, address, blockTo), address, blockTo)

#Returns the number of transactions in blockFrom - number of transactions in blockTo both sent FROM address
def out_transactionsDifferenceBetweenBlocks(df, address, blockFrom, blockTo):
    assert blockFrom < blockTo, 'blockFrom should be lower'
    assert len(address) == 42, 'address provided is incorrect'
    return out_degree(out_transaction_per_block(df, address, blockFrom), address, blockFrom) - out_degree(out_transaction_per_block(df, address, blockTo), address, blockTo)

#Returns the number of transactions in blockFrom - number of transactions in blockTo both sent FROM and TO address
def transactionsDifferenceBetweenBlocks(df, address, blockFrom, blockTo):
    assert blockFrom < blockTo, 'blockFrom should be lower'
    assert len(address) == 42, 'address provided is incorrect'
    return degree(transaction_per_block(df, address, blockFrom), address, blockFrom) - degree(transaction_per_block(df, address, blockTo), address, blockTo)

# R2 - Degree Burst: Si el in/out degree de un nodo i es mayor que \theta, ese nodo es sospechoso 

# R2.1: grado(A, B, G_in_out) & (G_in_out > t_gr_in+t_gr_out) --> hyp_malicioso(A)  
    
#Returns number of transaction TO and FROM address in the block number blockNumber
def degree(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    return df[(df['blockNumber']==blockNumber) & ((df['to']==address) | (df['from']==address))].count()['to']

# R2.2: grado_In(A, B, G_in) & (G_in > t_gr_in) --> hyp_malicioso(A)

#Returns number of transaction TO address in the block number blockNumber
def in_degree(df, address, blockNumber): 
    assert len(address) == 42, 'address provided is incorrect'
    return in_transaction_per_block(df, address, blockNumber).count()['from']
    
# R2.3: grado_out(A, B, G_out) & (G_out > t_gr_out) --> hyp_malicioso(A)  

#Returns number of transaction FROM address in the block number blockNumber
def out_degree(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    return out_transaction_per_block(df, address, blockNumber).count()['from']

# R3 - Balance Burst:  El balance que envía/recibe una cuenta A es mayor que \theta, ese nodo es sospechoso 

# R3.1: balance(A, B, Bal) & (Bal > t_bal) --> hyp_malicioso(A)   (in/out)

#Returns the different values the address SENT or RECEIVED in block number blockNumber
def balance(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    df_aux = transaction_per_block(df, address, blockNumber)
    return df_aux['value']
    
# R3.2: in_balance(A, B, in_Bal) & (in_Bal > t_bal) --> hyp_malicioso(A)

#Returns the different values the address RECEIVED in blockNumber
def in_balance(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    df_aux = in_transaction_per_block(df, address, blockNumber)
    return df_aux['value']

# R3.3: out_balance(A, B, out_Bal) & (out_Bal > t_bal) --> hyp_malicioso(A)

#Returns the different values the address SENT in blockNumber
def out_balance(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    df_aux = out_transaction_per_block(df, address, blockNumber)
    return df_aux['value']

# R4 - Gas Price Burst : Si el gas price que pone una cuenta en una tx es mayor que \theta, ese nodo es sospechoso 
# gasPrice(A, B, GP) & (GP > t_gp) --> hyp_malicioso(A) 

#Returns gasPrice in transactions RECEIVED by address belonged to block blockNumber
def in_gasPrice(df, address, blockNumber):
    assert len(address) == 42, 'address provided is incorrect'
    df_aux = in_transaction_per_block(df, address, blockNumber)
    return df_aux['gasPrice']


# R5 - Attractiveness: Si el nodo tiene un valor de attractiveness alto entonces ese nodo es sospechoso
# attractiveness(A,B1,ATT) & ATT > t_att --> hyp_malicioso(A)  

#We considered a neighbor of an account A in the graph, if the neighbor SENT a transaction to A

#Returns the nodes that are neighbors of the address in the time blockNumber
def neighborhood(df, blockNumber, address):
    assert len(address) == 42, 'address provided is incorrect'
    df_aux = in_transaction_per_block(df, address, blockNumber)
    return df_aux["from"].unique()

#Returns the nodes that are neighbors of the address in the time range fromBlockNumber..toBlockNumber
def neighborhood_from_to(df, fromBlockNumber, toBlocknumber, address):
    assert fromBlockNumber <= toBlocknumber, 'fromBlockNumber should be leq to toBlocknumber'
    assert len(address) == 42, 'address provided is incorrect'
    df_in = in_transactions(df, address)
    df_in_range = df_in[(df_in['blockNumber'] >= fromBlockNumber) & (df_in['blockNumber'] < toBlocknumber)]
    return df_in_range["from"].unique()

#Returns the attractivenes of the address
def attractiveness(df, fromBlockNumber, window, address):
    assert fromBlockNumber >= 0, 'invalid fromBlockNumber'
    assert fromBlockNumber >= window, 'window is lower than fromBlockNumber'
    assert len(address) == 42, 'address provided is incorrect'
    neighbors_in_t = neighborhood(df, fromBlockNumber, address)
    print("neighbors_in_t: ")
    print(neighbors_in_t)
    print('SIZE: ' + str(neighbors_in_t.size))
    if(neighbors_in_t.size == 0):
        return 0
    neighborhood_T_not_t = neighborhood_from_to(df, fromBlockNumber - window, fromBlockNumber, address) 
    print("neighborhood_T_not_t: ")
    print(neighborhood_T_not_t)
    print('SIZE: ' + str(neighborhood_T_not_t.size))
    neighborhood_T_with_t = neighborhood_from_to(df, fromBlockNumber - window, fromBlockNumber+1, address)
    print("neighborhood_T_with_t: ")
    print(neighborhood_T_with_t)
    print('SIZE: ' + str(neighborhood_T_with_t.size))
    not_changed_neighbors = np.intersect1d(neighbors_in_t, neighborhood_T_not_t).size
    print("not_changed_neighbors: ")
    print(not_changed_neighbors)
    print('SIZE: ' + str(neighborhood_T_with_t.size))
    return 1 - (not_changed_neighbors/neighborhood_T_with_t.size)
    
    
def blocks(df, address):
    df_result = df[df['from']==address]['blockNumber']
    return df_result
    
def gas_price(df, address, blockNumber, transactionHash):
    return df[df['from']==address and df['transactionHash']==transactionHash and df['blockNumber']==blockNumber]['gasPrice']




In [21]:
MG = nx.MultiDiGraph()
transaction_dataframe = transaction_dataframe[transaction_dataframe['to'] != 'None'] #This avoids 'None' as a node, so i'm erasing contract creations txs
MG = nx.from_pandas_edgelist(transaction_dataframe, 'from', 'to', edge_attr=['blockNumber','transactionHash', 'value', 'gasPrice'], create_using=nx.MultiGraph())
transaction_dataframe

,blockNumber,transactionHash,from,to,creates,value,gasLimit,gasPrice,gasUsed,isError
0,46147,0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84...,0xa1e4380a3b1f749673e270229993ee55f35663b4,0x5df9b87991262f6ba471f09758cde1c0fc1de734,None,31337,21000,50000000000000,21000,None
1,46169,0x19f1df2c7ee6b464720ad28e903aeda1a5ad8780afc2...,0xbd08e0cddec097db7901ea819a3d1fd9de8951a2,0x5c12a8e43faf884521c2454f39560e6c265a68c8,None,19900000000000000000,21000,909808707606,21000,None
2,46170,0x9e6e19637bb625a8ff3d052b7c2fe57dc78c55a15d25...,0x63ac545c991243fa18aec41d4f6f598e555015dc,0xc93f2250589a6563f5359051c1ea25746549f0d8,None,599989500000000000000,21000,500000000000,21000,None
3,46194,0xcb9378977089c773c074045b20ede2cdcc3a6ff562f4...,0x037dd056e7fdbd641db5b6bea2a8780a83fae180,0x7e7ec15a5944e978257ddae0008c2f2ece0a6090,None,100000000000000000000,21000,1000000000000,21000,None
4,46205,0x570ce19176bd0002b04a9179309129bbdaf0c4252ffe...,0x3f2f381491797cc5c0d48296c14fd0cd00cdfa2d,0x4bd5f0ee173c81d42765154865ee69361b6ad189,None,803989500000000000000,21000,500000000000,21000,None
...,...,...,...,...,...,...,...,...,...,...
1674257,999999,0x4fa879b491e0779fc035758ec77b93c4e51d528d65b6...,0x5c51467399bc655f0cc6db88df15946717534633,0x32be343b94f860124dc4fee278fdcbd38c102d88,None,2041506130000000000,21000,60000000000,21000,None
1674258,999999,0x1bea59827ab153b20cee79890d221a80fa6a04e552d6...,0x055d9d7ec193d1e062c6ec4fa80ef89b5c1258f4,0x32be343b94f860124dc4fee278fdcbd38c102d88,None,1082068640000000000,21000,60000000000,21000,None
1674259,999999,0x73e87db1108a2aa852f48e088ca1a2771f9b7c18af8d...,0x8e68c0c9b5275fa684291304af9cafe6ceaf2772,0x26016a2b5d872adc1b131a4cd9d4b18789d0d9eb,None,100000000000000000,90000,50000000000,21000,None
1674260,999999,0x337a5e90b73f44ffebea73cb3d97738c524f63e1032b...,0x2a65aca4d5fc5b5c859090a6c34d164135398226,0x5275c3371ece4d4a5b1e14cf6dbfc2277d58ef92,None,1050440880000000000,90000,50000000000,21000,None


In [ ]:
# # in_transaction_per_block(transaction_dataframe, '0x32be343b94f860124dc4fee278fdcbd38c102d88', 999998)
# np.intersect1d(neighborhood(transaction_dataframe, '0x32be343b94f860124dc4fee278fdcbd38c102d88', 999998), neighborhood(transaction_dataframe, '0x32be343b94f860124dc4fee278fdcbd38c102d88', 999998))
# in_transaction_per_block(transaction_dataframe, '0x32be343b94f860124dc4fee278fdcbd38c102d88', 999998)
# print('0x32be343b94f860124dc4fee278fdcbd38c102d88')
# attractiveness(transaction_dataframe, 999999, 999998, '0x32be343b94f860124dc4fee278fdcbd38c102d88')

# print('0x7de5aba7de728950c92c57d08e20d4077161f12f')
# attractiveness(transaction_dataframe, 835037, 835036, '0x7bd4583297fced654fd5ad77fd1e99f648ef70c4')


# neighborhood(transaction_dataframe, 999999, '0x32be343b94f860124dc4fee278fdcbd38c102d88')
# def neighborhood(df, address, blockNumber):
#     df_aux = in_transaction_per_block(df, address, blockNumber)
#     return df_aux["from"].unique()
# len('0xB8c77482e45F1F44dE1745F52C74426C631bDD52')

# df = transaction_dataframe.groupby('to').count().sort_values(by='blockNumber')
# df = transaction_dataframe[transaction_dataframe['to'] == '0x7bd4583297fced654fd5ad77fd1e99f648ef70c4']
# df

MG.get_edge_data('0xa1e4380a3b1f749673e270229993ee55f35663b4','0x5df9b87991262f6ba471f09758cde1c0fc1de734')[0] == MG.get_edge_data('0xa1e4380a3b1f749673e270229993ee55f35663b4','0x5df9b87991262f6ba471f09758cde1c0fc1de734')[0]

In [ ]:
# import infura


# ifr = infura.Client(
#     project_id='3c62e1e38e234dd8b61022ad73661486',
#     project_secret='c6f80547b6ff4da9bc649b3945f9546b',
#     network='mainnet',
#     cache_expire_after=5,
# )


# block_number = ifr.eth_get_block_number()

# block = ifr.eth_get_block_by_number(block_number, show_transaction_details = True)
# block


# blocks(address) : Conj<blockNumber>
# inDegree(address, blockNumber) : num
# outDegree(address, blockNumber) : num
# transacciones(address) : Conj<(Transacciones, blockNumber)>
# transacciones(address, blockNumber) : Conj<Transacciones>
# gasPrice(tx, address, blockNumber) : num
# numTransacciones(address, blockNumber) : num
# balance(address, blockNumber) : num
# vecinos(address, blockNumber) : Conj<nodo>
# contratosCreados(address) : Conj<address>
# invocacionesDe(address) : Conj<tx> (data != null)
# invocacionesA(address) : Conj<tx> (data != null)
# esInvocacion(tx) : bool
# esTransferenciaDeValor(tx) : bool
# esCreacionDeContrato(tx) : bool
# value(tx) : num
# esContrato(address) : bool
# esEOA(address) : bool
# diferenciaDeTransacciones(address, blockNumber, blockNumber) : num


# Nodo: {address, balance, nonce, data}
# Eje: {from: address, to: address, value, nonce, data} (info relevante de la tx)
# Del bloque Bk..Bn (k < n)
# Por cada transacción tx en Bi:
# Se crea un eje e dirigido
# Si no existen se crean los nodos nf, nt :=  tx.from, tx.to 
# (habría que ver los casos bordes donde tx.from.balance es 0 ya que lo creamos)
# e.from = tx.from, e.to = tx.to, e.blockNum = Bi.number
# 	Si el tx.to = null es una creación de contrato. 
# 		e.to := tx.receipt.contractAddress
# 		nt := tx.receipt.contractAddress
# 	Si data != vacio y tx.to != null es una invocación eje.data =  tx.data
# 	Si value > 0 es una transferencia de valor 
# e.value = tx.value, Nt.balance += e.value

In [ ]:
#Edges from address
MG['0x32be343b94f860124dc4fee278fdcbd38c102d88']

In [ ]:
transaction_dataframe[(transaction_dataframe['from'] == '0x32be343b94f860124dc4fee278fdcbd38c102d88')]

In [ ]:
pd.options.display.max_colwidth = 100
transaction_dataframe[(transaction_dataframe['to'] == 'None') & (transaction_dataframe['creates'] == 'None')]
